In [1]:
#Upgrade to tensorflow2.x
!pip install --upgrade tensorflow

     |████████████████████████████████| 421.8MB 38kB/s 
     |████████████████████████████████| 3.9MB 49.0MB/s 
     |████████████████████████████████| 450kB 56.3MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
import tensorflow as tf

import sklearn
from sklearn.metrics import jaccard_score

from tensorflow.keras import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D, concatenate
from tensorflow.keras.layers import Dense, Dropout, Activation, Reshape, Flatten
# from tensorflow.keras.utils import np_utils
# from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras.callbacks import CSVLogger
# from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics

import numpy as np
import os
import cv2
import imutils
import random
from PIL import Image
import matplotlib.pyplot as plt
import requests
import shutil
import json
import pandas as pd

In [0]:
print(tf.__version__)

2.1.0


In [0]:
print(os.getcwd())



/content


In [2]:
from google.colab import drive
drive.mount('/content/drive/')
!ls /content/drive/My\ Drive/JSTP

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
 20191029_125129.mp4   test_frames   train_frames   val_frames
'JSTP slide.mp4'       test_masks    train_masks    val_masks


['20191029_125129.mp4',
 'JSTP slide.mp4',
 'train_masks',
 'test_masks',
 'train_frames',
 'val_masks',
 'val_frames',
 'test_frames']

Copying file://adc.json [Content-Type=application/json]...
AccessDeniedException: 403 The project to be billed is associated with an absent billing account.
cat: /train_frames: No such file or directory


In [3]:
DATA_PATH = 'drive/My Drive/JSTP'

train_frame_path = DATA_PATH+'/train_frames/'
train_mask_path = DATA_PATH+'/train_masks/'

val_frame_path = DATA_PATH+'/val_frames/'
val_mask_path = DATA_PATH+'/val_masks/'

img_path = os.listdir(train_frame_path)
mask_path = os.listdir(train_mask_path)
random.shuffle(img_path)
random.shuffle(mask_path)

train_img = np.zeros((50, 640, 360, 1)).astype('float')
train_mask = np.zeros((50, 640, 360, 1)).astype('float')

for i in range(0, 50): #initially from 0 to 16, c = 0. 
        
    train_img1 = cv2.imread(train_frame_path+img_path[i],0)/255.
    
    if train_img1.shape != (1920,1080):
        train_img1 = imutils.rotate_bound(train_img1, 90)
        
    train_img1 =  cv2.resize(train_img1, (360, 640))# Read an image from folder and resize
    train_img1 = train_img1.reshape(640, 360, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]
    train_img[i] = train_img1 #add to array - img[0], img[1], and so on.
        
    train_mask1 = cv2.imread(train_mask_path+mask_path[i], cv2.IMREAD_GRAYSCALE)/255.
    if train_mask1.shape != (1920,1080): train_mask1 = imutils.rotate_bound(train_mask1, 90)

    train_mask1 = cv2.resize(train_mask1, (360, 640))

    train_mask1 = train_mask1.reshape(640, 360, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]

    train_mask[i] = train_mask1
    
# Validation set
    
img_path = os.listdir(val_frame_path)
mask_path = os.listdir(val_mask_path)
val_img = np.zeros((10, 640, 360, 1)).astype('float')
val_mask = np.zeros((10, 640, 360, 1)).astype('float')

for i in range(0, 10): #initially from 0 to 16, c = 0. 
        
    val_img1 = cv2.imread(val_frame_path+img_path[i],0)/255.
    
    if val_img1.shape != (1920,1080): 
        val_img1 = imutils.rotate_bound(val_img1, 90)
        
    val_img1 =  cv2.resize(val_img1, (360, 640))# Read an image from folder and resize
    val_img1 = val_img1.reshape(640, 360, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]
    val_img[i] = val_img1 #add to array - img[0], img[1], and so on.
        
    val_mask1 = cv2.imread(val_mask_path+mask_path[i], cv2.IMREAD_GRAYSCALE)/255.
    if val_mask1.shape != (1920,1080): val_mask1 = imutils.rotate_bound(val_mask1, 90)

    val_mask1 = cv2.resize(val_mask1, (360, 640))

    val_mask1 = val_mask1.reshape(640, 360, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]

    val_mask[i] = val_mask1
    
'''
def data_gen(img_folder, mask_folder):
    n = os.listdir(img_folder) #List of training images
    random.shuffle(n)
    
    print(n)
    
    img = np.zeros(50, 360, 640, 1).astype('float')
    mask = np.zeros(50, 360, 640, 1).astype('float')
    print(img,mask)
    for i in range(0, 50): #initially from 0 to 16, c = 0. 
        
        train_img = cv2.imread(img_folder+n[i])/255.
        train_img =  cv2.resize(train_img, (360, 640))# Read an image from folder and resize
      
        img[i] = train_img #add to array - img[0], img[1], and so on.
        
        print(img[i])
        
        train_mask = cv2.imread(mask_folder+n[i], cv2.IMREAD_GRAYSCALE)/255.
        train_mask = cv2.resize(train_mask, (360, 640))
        train_mask = train_mask.reshape(360, 640, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]

        mask[i] = train_mask
    
    yield img, mask
    
'''


"\ndef data_gen(img_folder, mask_folder):\n    n = os.listdir(img_folder) #List of training images\n    random.shuffle(n)\n    \n    print(n)\n    \n    img = np.zeros(50, 360, 640, 1).astype('float')\n    mask = np.zeros(50, 360, 640, 1).astype('float')\n    print(img,mask)\n    for i in range(0, 50): #initially from 0 to 16, c = 0. \n        \n        train_img = cv2.imread(img_folder+n[i])/255.\n        train_img =  cv2.resize(train_img, (360, 640))# Read an image from folder and resize\n      \n        img[i] = train_img #add to array - img[0], img[1], and so on.\n        \n        print(img[i])\n        \n        train_mask = cv2.imread(mask_folder+n[i], cv2.IMREAD_GRAYSCALE)/255.\n        train_mask = cv2.resize(train_mask, (360, 640))\n        train_mask = train_mask.reshape(360, 640, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]\n\n        mask[i] = train_mask\n    \n    yield img, mask\n    \n"

In [0]:
model = Sequential()


#1st Layer
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(MaxPooling2D(pool_size=2))

#1st Layer
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(MaxPooling2D(pool_size=2))

#1st Layer
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(MaxPooling2D(pool_size=2))


#Upsampling Part
model.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest'))
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))

#Upsampling Part
model.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest'))
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))

#Upsampling Part
model.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest'))
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))

#Upsampling Part
model.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest'))
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))

#Upsampling Part
model.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest'))
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(64, (17, 6), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(64, (17, 10), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(64, (17, 6), activation='relu', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))

model.add(Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer='he_uniform', input_shape=(640, 360, 1)))

In [6]:
(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 624, 351, 64)      10944     
_________________________________________________________________
batch_normalization_18 (Batc (None, 624, 351, 64)      256       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 608, 342, 64)      696384    
_________________________________________________________________
batch_normalization_19 (Batc (None, 608, 342, 64)      256       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 304, 171, 64)      0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 288, 162, 64)      696384    
_________________________________________________________________
batch_normalization_20 (Batc (None, 288, 162, 64)     

In [7]:
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

with strategy.scope():
  model = model
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=['accuracy'])

model.fit(
    train_img.astype(np.float32), train_mask.astype(np.float32),
    epochs=17,
    steps_per_epoch=60,
    validation_data=(val_img.astype(np.float32), val_mask.astype(np.float32)),
    validation_freq=17
)

model.save_weights('./sidewalk_weight.h5', overwrite=True)

INFO:tensorflow:Initializing the TPU system: 10.1.124.226:8470


INFO:tensorflow:Initializing the TPU system: 10.1.124.226:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


ValueError: ignored

In [0]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Initializing the TPU system: 10.93.215.194:8470


INFO:tensorflow:Initializing the TPU system: 10.93.215.194:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [0]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])
  
train_dataset, test_dataset = get_dataset()

model.fit(train_dataset,
          epochs=5,
          validation_data=test_dataset)

In [13]:
!git remote add gdrive /drive/My\ drive/github_project/.git

fatal: not a git repository (or any of the parent directories): .git
